In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact
from scipy.stats import norm
from collections import Counter
from scipy.stats import binomtest
from scipy.stats import spearmanr,pearsonr

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

def parse_table(i):
    return [int(x) for x in i.replace("[", "").replace("]", "").split(",")]

d_abrev = {"LiangSteinNeuron":"FC exc. neur.", "FetalChondrocytes":"F chond.", "SertoliMale":"FG sertoli", "preGC_IIaFemale":"FG preGC IIa",\
          "NeuralFemale":"FG neur.", "FetalGonadImmuneFemale":"FG immune", "VIP":"AC VIP inh. neur.", "LiangSteinProgenitor":"FC prog.",\
          "AdultHeartVentricularCardiomyocyte":"AH cardiomyo.", "AdultLoopOfHenle":"AK loop of henle", "FetalBrainNeurGlioblast_CB_VZ":"FCB glioblast",\
         "AdultProximalTubule":"AK prox. tub.", "FetalLeydigMale":"FG leydig", "SST":"AC SST inh neur.", "KosoyRoussosControlMicroglia":"AC microglia",\
         "FetalBrainFloorPlate":"FB fl. plate", "FetalArterialECs":"FH endoth.", "ASCT":"AC astro.", "FetalBrainCOP":"FB COP",\
         "AMY":"AA neur.", "PVALB":"AC PVALB inh neur.", "ITL23":"AC L2-3 IT neur.", "FetalBrainNeurCB_GNP_IPC_1":"FB inter. prog.", "FetalBrainNeurDAergic":"FB DA neur.",\
          "OGC":"AC Oligo.", "D1Pu":"AP D1 inh neur.", "FetalBrainNeurSerotonergic":"FB 5-HT neur.", "FetalBrainNeurDRG_2":"FS DRG neur.",\
          "FetalHeartPericytes":"FH peri.", "FetalHeartEndocardium":"FH endocard.", "FetalHeartCardiacFibroblasts":"FH fibro.", "FetalBrainNeurPurkinje_6":"FCB Purk. inh neur.",\
          "AdultHeartSmoothMuscle":"AH smooth musc.", "FetalBrainRoofPlate":"FB ro. plate"}

gocc = pd.read_csv("../DPSC_CNCC/GOCC_AccelEvol_Input.txt", sep= "\t")
d_CC = {}

for index, row in gocc.iterrows():
    d_CC[row["Term"]] = row["Genes"].split(";")

In [ ]:
v2 = pd.read_csv("TF_Data/MA1643.1_NFIB_400_LiangSteinNeuron_Fixed.csv.gz")
vv2 = pd.read_csv("TF_Data/MA1643.1_NFIB_400_LiangSteinNeuron_Poly.csv.gz")
v2 = v2[(v2["1"] > 500) | ((v2["2"] > 500) & (v2["4"] > 500))]
vv2 = vv2[(vv2["1"] > 500) | ((vv2["2"] > 500) & (vv2["3"] > 500))]

be = list(pd.read_csv("BlacklistTE_Variants.txt")["Position"])
v2 = v2[~v2["Position"].isin(be)]
vv2 = vv2[~vv2["Position"].isin(be)]

v2_fore = v2[v2["abs logfc"] > 0.25]

v2_fore["Chrom"] = [x.split(":")[0] for x in v2_fore["Position"]]
v2_fore["Pos1"] = [int(x.split(":")[1]) for x in v2_fore["Position"]]
v2_fore["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2_fore["Position"]]
v2_fore = v2_fore[["Chrom", "Pos1", "Pos2"]]

#Need to change 
v2_fore.to_csv("TCF12_AbsG0.25_Fixed_LSN_FilterNewTEs.bed", sep = "\t", header = None, index = None)

v2["Chrom"] = [x.split(":")[0] for x in v2["Position"]]
v2["Pos1"] = [int(x.split(":")[1]) for x in v2["Position"]]
v2["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2["Position"]]

#Set to 0.05 for MEF2A
#Change 0.025 to 0.1 for NFIB and other TFs with large effects on CA
v2_back = pd.concat([v2_fore, v2[v2["abs logfc"] < 0.1]])
v2_back = v2_back[["Chrom", "Pos1", "Pos2"]]
v2_back.to_csv("TCF12_AbsG0.25_Fixed_Background_New_preGC_FilterNewTEs.bed", sep = "\t", header = None, index = None)

In [ ]:
#Can change various things in this cell to make plots for different TFs, restrict to synaptic genes, etc.
#Enrichment for Rxra gets even stronger at 550, 600
#Plot a TF by changing the ID

v2 = pd.read_csv("TF_Data/MA1643.1_NFIB_400_LiangSteinNeuron_Fixed.csv.gz")
vv2 = pd.read_csv("TF_Data/MA1643.1_NFIB_400_LiangSteinNeuron_Poly.csv.gz")
v2 = v2[(v2["1"] > 500) | ((v2["2"] > 500) & (v2["4"] > 500))]
vv2 = vv2[(vv2["1"] > 500) | ((vv2["2"] > 500) & (vv2["3"] > 500))]

be = list(pd.read_csv("BlacklistTE_Variants.txt")["Position"])
v2 = v2[~v2["Position"].isin(be)]
vv2 = vv2[~vv2["Position"].isin(be)]

#Can comment or uncomment to change synaptic restriction
syngo = pd.read_csv("../Cell_Type_Prop/Cell_Type_Prop/syngo_genes.csv")
syngo = syngo["hgnc_symbol"]

#v2 = v2[~v2["NearestGene"].isin(syngo)]
#vv2 = vv2[~vv2["NearestGene"].isin(syngo)]

#v2 = v2[v2["abs logfc"] > 0.25]
#vv2 = vv2[vv2["abs logfc"] > 0.25]

#v2 = v2[v2["SpecSup447"] > 250]
#vv2 = vv2[vv2["SpecSup447"] > 250]

vv2 = vv2[vv2["Human ref"] != vv2["Chimp ref"]]
cuttt = 0.6

z = list(vv2["abs logfc"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

vvv = prepare_alpha(v2, vv2, stat = "abs logfc")
#vvv["PhyloP"] = -vvv["PhyloP"]
alpha = compute_alpha_cutoff(vvv, plot = True, window = [0.2, 0.8], cutoff = cutoff, title = "NFIB binding sites in fetal cortical exc. neurons")
plt.xlabel("Predicted absolute log$_{2}$ fold-change in CA", fontsize = 20)
plt.ylabel("Density", fontsize = 20)
print(alpha)
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)

plt.title("")
plt.xlabel("")
plt.ylabel("")
#plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
#Across various cutoffs, we observe that when the sign of the motif difference is consistent with the sign of the logfc
#Is consistent with the sign of the RNA log fold-change, we observe a significant bias toward downregulation of the gene
#This holds across cutoffs for abs logfc, padj for RNA, and Top20th
cutoff_lsn = (531.89703 + 520.9438)/2

v2ps = v2p[v2p["abs logfc"] > 0.1]
x = pd.read_csv("Agoglia_Fraser_2021_DESeq2_HumChp_Hybrid_CS_D100_chpr.txt", sep = "\t").dropna()
x = x[x["padj"] < 0.05]
x = x[x["Gene"].isin(v2p["NearestGene"])]

v2ps["Sign1"] = np.sign(v2ps["logfc"])
v2ps["Dif"] = v2ps["1"] - v2ps["2"]
v2ps["Sign2"] = np.sign(v2ps["Dif"])
v2ps["Sign"] = v2ps["Sign1"]*v2ps["Sign2"]
v2ps = v2ps[v2ps["Sign"] < 0]

v2ps = v2ps[v2ps["NearestGene"].isin(x["Gene"])]
v2ps = v2ps.set_index("NearestGene").join(x.set_index("Gene"))
v2ps = v2ps[(v2ps["allele1_pred_counts"] > cutoff_lsn) | (v2ps["allele2_pred_counts"] > cutoff_lsn)]
v2ps["Sign3"] = np.sign(v2ps["log2FoldChange"])
v2ps["Sign4"] = v2ps["Sign3"]*v2ps["Sign1"]

#Testing whether there is more agreement in NFIB effect and RNA l2fc than expected by chance
agree = v2ps[v2ps["Sign4"] < 0].shape[0]
disagree = v2ps[v2ps["Sign4"] > 0].shape[0]
print(binomtest(agree, agree + disagree))

v2ps = v2ps[v2ps["Sign4"] < 0]
v2ps_nd = v2ps.drop_duplicates("pvalue")
down = v2ps_nd[v2ps_nd["log2FoldChange"] < 0].shape[0]
up = v2ps_nd[v2ps_nd["log2FoldChange"] > 0].shape[0]
print(binomtest(down, down + up))

#down = v2ps[v2ps["log2FoldChange"] < 0].shape[0]
#up = v2ps[v2ps["log2FoldChange"] > 0].shape[0]
#print(binomtest(down, down + up))

In [ ]:
#barplot of ASE values
to_plot = v2ps_nd.sort_values("log2FoldChange")
out = []
for index, row in to_plot.iterrows():
    #out.append(["$_{" + index + "}$", row["log2FoldChange HumChp"], "Hybrid cortical organoid"])
    #out.append(["$_{" + index + "}$", row["log2FoldChange HumChp Adult"], "Adult MTG L2/3 IT"])
    if row["log2FoldChange"] < 0:
        x = "Down human"
    else:
        x = "Up human"
    out.append(["$_{" + index + "}$", row["log2FoldChange"], x])
to_plot = pd.DataFrame(out)
to_plot.columns = ["Gene", "Log$_{2}$ fold-change", "Direction"]
sns.set(font_scale = 1.5)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (9, 4))
sns.set_style("white")
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
t_ax = sns.barplot(data = to_plot, x = "Gene", y = "Log$_{2}$ fold-change", hue = "Direction")


c = 0
for patch in t_ax.patches:
    if c < 23:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    else:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    c += 1
plt.title("ASE for synaptic genes near NFIB binding sites in day 100 cortical organoids", size = 15)
plt.xticks(rotation=90)
plt.ylabel("Log$_{2}$ fold-change human/chimp", size = 14)
plt.legend([], [], frameon = False)

In [ ]:
#Organizing for biorender plot using SYNGO annotations + looking up on Uniprot
local = {"Presynapse":["ADORA2B", "CACNA2D3", "CADM1", "CD47", "CPEB2", "CTNNB1", "GRIN2A", "OTOF", "PCDH9", "PRKCA", "PTPRN2", "PTPRS", "RIMBP2", "RPL13A", "RPL38", "STX7", "SV2C", "VLDLR", "VPS35"], "Postsynapse":["ARF1", "ARHGAP32", "CADM1", "CPEB2", "CTNNB1", "GRIN2A", "LASP1", "PCDH9", "RAB4A", "RPL13A", "RPL38", "SIPA1L3", "SRGAP2", "VLDLR", "VPS35"], "Membrane":["CDH23", "FKBP1A", "NCAM2", "RAB3C"]}

In [ ]:
for i in local["Membrane"]:
    print(i)

In [ ]:
v2ps_nd.loc[local["Membrane"]][v2ps_nd["log2FoldChange"] > 0]

In [ ]:
syn = pd.read_csv("syngo_annotations.csv")
syn[syn["hgnc_symbol"].isin(v2ps_nd.index)].tail(50)

In [ ]:
v2ps_nd.loc[local["Postsynapse"]]

In [ ]:
#Barplot in cortical organoids
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))
t_ax = sns.barplot({"Upregulated\nin human":7, "Downregulated\nin human":23}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = {"Upregulated\nin human":new_palette["Human"], "Downregulated\nin human":new_palette["Chimp"]})
for patch in t_ax.patches:
    print(patch.get_x() + patch.get_width()/2)
c = 0
for patch in t_ax.patches:
    if c < 1:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    elif c == 1:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    c += 1
plt.ylabel("Number of genes")
plt.title("Day 100 cortical organoids")

In [ ]:
#Permutation test to confirm GREAT results
real = np.mean(v2p["abs logfc"])
size = v2p.shape[0]

fakes = []
for i in range(10000):
    np.random.seed(i)
    fake = np.mean(v2.sample(n = size, replace = False)["abs logfc"])
    fakes.append(fake)
print(norm.sf((real - np.mean(fakes))/np.std(fakes)))



In [ ]:
sns.kdeplot(fakes, fill = True, color = "#40A94D")
plt.vlines(x = real, ymin = 0, ymax = 100, color = "#F42FF5")
plt.ylim(0, 100)
plt.xlabel("Mean abs logfc of gene set", size = 14)
plt.ylabel("Density of random subsets", size = 14)
plt.title("MEF2A binding sites near synaptic assembly regulators", size = 18)

In [ ]:
### Create inputs to GREAT to test whether sites with large abs logfc are enriched near certain types of genes

#Change 0.1 to 0.25 for NFIB and other TFs with large effects on CA
v2_fore = v2[v2["abs logfc"] > 0.1]

v2_fore["Chrom"] = [x.split(":")[0] for x in v2_fore["Position"]]
v2_fore["Pos1"] = [int(x.split(":")[1]) for x in v2_fore["Position"]]
v2_fore["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2_fore["Position"]]
v2_fore = v2_fore[["Chrom", "Pos1", "Pos2"]]

#Need to change 
v2_fore.to_csv("ESR1_AbsG0.1_Fixed_preGC_FilterNewTEs.bed", sep = "\t", header = None, index = None)

v2["Chrom"] = [x.split(":")[0] for x in v2["Position"]]
v2["Pos1"] = [int(x.split(":")[1]) for x in v2["Position"]]
v2["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2["Position"]]

#Set to 0.05 for MEF2A
#Change 0.025 to 0.1 for NFIB and other TFs with large effects on CA
v2_back = pd.concat([v2_fore, v2[v2["abs logfc"] < 0.025]])
v2_back = v2_back[["Chrom", "Pos1", "Pos2"]]
v2_back.to_csv("ESR1_AbsG0.1_Fixed_Background_New_preGC_FilterNewTEs.bed", sep = "\t", header = None, index = None)

In [ ]:
### COPY OF CELL ABOVE BUT RECORDING WHAT WE DID FOR NFIB ###

#v2 = v2[(v2["allele1_pred_counts"] > 350) | (v2["allele1_pred_counts"] > 350)]
v2_fore = v2[v2["abs logfc"] > 0.25]

v2_fore["Chrom"] = [x.split(":")[0] for x in v2_fore["Position"]]
v2_fore["Pos1"] = [int(x.split(":")[1]) for x in v2_fore["Position"]]
v2_fore["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2_fore["Position"]]
v2_fore = v2_fore[["Chrom", "Pos1", "Pos2"]]
v2_fore.to_csv("NFIB_AbsG0.25_Fixed_LSN_FilterNewTEs.bed", sep = "\t", header = None, index = None)

v2["Chrom"] = [x.split(":")[0] for x in v2["Position"]]
v2["Pos1"] = [int(x.split(":")[1]) for x in v2["Position"]]
v2["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2["Position"]]

#Set to 0.05 for MEF2A
v2_back = pd.concat([v2_fore, v2[v2["abs logfc"] < 0.1]])
v2_back = v2_back[["Chrom", "Pos1", "Pos2"]]
v2_back.to_csv("NFIB_AbsG0.25_Fixed_Background_New_LSN_FilterNewTEs.bed", sep = "\t", header = None, index = None)

vv2["Chrom"] = [x.split(":")[0] for x in vv2["Position"]]
vv2["Pos1"] = [int(x.split(":")[1]) for x in vv2["Position"]]
vv2["Pos2"] = [int(x.split(":")[1]) + 1 for x in vv2["Position"]]
v2_back = pd.concat([v2_fore, vv2])
v2_back = v2_back[["Chrom", "Pos1", "Pos2"]]
v2_back.to_csv("NFIB_AbsG0.25_Fixed_Background_Poly_LSN_FilterNewTEs.bed", sep = "\t", header = None, index = None)